### 抓露天小項

In [6]:
import requests
from bs4 import BeautifulSoup

headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}


res = requests.get('http://class.ruten.com.tw/category/' , headers=headers)
soup = BeautifulSoup(res.text )
#soup = BeautifulSoup(res.text, "lxml", from_encoding="GB18030")

#無法顯示我猜是與修潔的版本問題
#原始資料內含@ 取代
q=[]
for ens in soup.select('div.rt-grid-1-5'):          
    word=(ens.select('a')[0].text.replace('@','').encode('latin-1').decode('utf-8'))
#     print word
    q.append(word.strip())

# print q
# for i in q:
#     print i
    

In [18]:
print type(q[1])

<type 'unicode'>


### 從露天小項查fb洞察報告

In [12]:
#token,cookie需定期更新
#資料是屬於隨時變動的



import requests
import json
import re
import time
from random import randint


# q=[u'筆記型電腦'],用來裝篩選過後的q值
newq={}
###上面已抓露天分類小項做出興趣的list q
#查詢的東西
#q = '蛋糕'
gender = ['',1,2]


# 年齡18~65  (暫定最大為25)
agerange=[]
Max=25
Min=18
for i in xrange(0,(Max-Min)/5+1):
    age=Min+i*5
    agerange.append(age)
# print agerange


#每一個metric代表畫面的一個欄位# 9 是指教育程度內資料# 30廣告受眾總人數# 3男女分配
# 17生活型態# 27感情狀況# 15職稱
#metric='9'
metric={u'男女分配':3,u'教育程度':9,u'職稱':15,u'生活型態':17,u'感情狀況':27,u'廣告受眾男女人數':30}


# ty=[ch,relationship]  


genderD = [u'所有', u'女性' ,u'男性']


#搜尋興趣代碼
###找js裡面targetingsearch裡面response是顯示idxxxxxxxx的headers複製Request URL:中間有一段token當token1
token1 ='EAAJ6ne6FIaEBAIgygcSGsDiB6icJ2Rf1T3mllrL6ZAKUuc6CpE2V7cnuXuqtpZCZAZBNX6Lvqlt3Vjm9ZAeLtrJTGk61ZCI351z7wWnjwNfQqhYCZBppcEwIcoc8RZBzE0xwoSnZCCSDPgZBBWQfZBBzMqsygui6sRl3pMzi1VH0DQpZAndHgIbcQzmJ'

header ={
    #第二次要加cookie
    'cookie':'datr=61_eVu2VCU02VgYNV8CL-IOb; _ga=GA1.2.532856099.1463722397; locale=zh_TW; sb=-xELV_JPo4Ol3GvSoynCxrcG; c_user=100012327361026; fr=0UXv5NOrWNaBuli1H.AWX2Anzbiw5rLBbdFaPzEO4rskY.BW3mAA.MH.AAA.0.0.AWXw73kZ; xs=89%3AWPqBdpexgBSC_A%3A2%3A1464831096%3A-1; csm=2; s=Aa7YWGLU4W35-Nsy.BXT4x4; pl=n; lu=ggl4BbKMwx4rW_U2bbqpawvQ; act=1464831124605%2F6; presence=EDvF3EtimeF1464831127EuserFA21B12327361026A2EstateFDutF1464831127435CEchFDp_5f1B12327361026F2CC; p=-2',
    'user-agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
}

#過濾興趣，並找出對硬的qid及name
# find_qid_actualName(q,token1,header)
find_qid_actualName(token1,header)


# print newq.keys()


#不希望一直跳出迴圈，作業量大的往內圈移動
###  j迴圈控制興趣
for j in newq.keys():
        qid=j
        name_interest=newq[qid]
        time.sleep(randint(0,5))
        ###  i迴圈控制性別
        for i in range(0,len(gender)):

            ###  n迴圈控制年齡
            for n in xrange(0,len(agerange)):
                ageMin=agerange[n]
                for m in xrange(n,len(agerange)-1):
                    ageMax=agerange[m+1]
                    interest ='&age[0]={}&age[1]={}&country[0]=TW&gender={}&interests[0]={}'.format(ageMin,ageMax,gender[i],qid)
                    #logger_session_id=也需要換
                    user1='admarket_id=6040879766077&logger_session_id=ee5882ad709ae5367dbcbee6b04231116bfd8521&__user=100012327361026&__a=1&__dyn=aK4XK4GgPwHz9HpUS2q2K9yaGujy8jXWqxiE9EoUaEOVEoyUnwExeqq1yxZoK3eu4FpHVoG4UdF8uwh9UcU89o-2it1ScwJwj8gCGch8G9x24o&__req=8'

                    
                    #提出不同表格中的值
                    for ele in metric.keys():
                        metric_value=metric[ele]
                        url2='https://www.facebook.com/ads/audience-insights/query/?dpr=1{}&metrics[0]={}&{}&__be=0&__pc=PHASED%3ADEFAULT&__rev=2365621'.format(interest,metric_value,user1)

#                         print url2
                        res2=requests.get(url2 ,headers = header)
#                         print (res2.text)
                        
#                         print ageMin,ageMax,gender[i],qid,metric_value
#                         print '-------------------------------------'
                        ###------------------------------------------------
                        #下一步為就不同Metric分別提出值
                        
                        text1=res2.text
                        text2=text1.split('"payload":')[1]
                        text3=text2.split(',"bootloadable"')[0]
                        # print text2
                        # print type(text2)
                        text3=json.loads(text3)
                        # print text3
                        
                        #找教育程度的人數  9
                        if(metric_value==9):
                            edu_info()
                        #職稱  15
                        elif(metric_value==15):
                            emp_name()
                        #感情狀況 27
                        elif(metric_value==27):
                            relation()
                        #男女比例 30
                        elif(metric_value==30):
                            gnum=text3['30']['data']['audience'][0]
                            bnum=text3['30']['data']['audience'][1]
                            print ageMin,ageMax,gender[i],qid,name_interest ,u'女:',gnum
                            print ageMin,ageMax,gender[i],qid,name_interest ,u'男:',bnum
                        
                            
                           

18 23  6003473371887 筆記型電腦 大專院校 168323.803366
18 23  6003473371887 筆記型電腦 高中 57280.2068395
18 23  6003473371887 筆記型電腦 研究所 4596.00347524
18 23  6003473371887 筆記型電腦 女: 250000
18 23  6003473371887 筆記型電腦 男: 300000
18 23  6003473371887 筆記型電腦 藝術、娛樂、運動和媒體 4663.17589605
18 23  6003473371887 筆記型電腦 行政管理 4874.38514836
18 23  6003473371887 筆記型電腦 零售 846.24742159
18 23  6003473371887 筆記型電腦 保健與醫學 1487.54429576
18 23  6003473371887 筆記型電腦 軍隊軍事 1066.6243543
18 23  6003473371887 筆記型電腦 教育和圖書館 1276.92691293
18 23  6003473371887 筆記型電腦 製造 3818.69148992
18 23  6003473371887 筆記型電腦 護士 211.561855397
18 23  6003473371887 筆記型電腦 商業和金融操作 1276.92691293
18 23  6003473371887 筆記型電腦 資訊科技和技術 1692.49484318
18 23  6003473371887 筆記型電腦 建築與工程 1487.54429576
18 23  6003473371887 筆記型電腦 管理 5318.22733051
18 23  6003473371887 筆記型電腦 運輸與搬運 1487.54429576
18 23  6003473371887 筆記型電腦 防治服務 423.123710795
18 23  6003473371887 筆記型電腦 業務 3830.7807388
18 23  6003473371887 筆記型電腦 社區和社會服務 846.24742159
18 23  6003473371887 筆記型電腦 個人護理 2331.58794803
18

In [1]:

#過濾興趣，並找出對硬的qid及name
def find_qid_actualName(token1,header):
        ###  k迴圈控制興趣
    # for k in range(0,len(q)):
    for k in range(0,1):
        
        url ='https://graph.facebook.com/v2.5/search?_reqName=path%3A%2Fsearch&_reqSrc=AdsDaoGraphDataProvider&access_token={}&endpoint=%2Fsearch&locale=zh_TW&method=get&pretty=0&q={}&type=adInterest'
        #竟然連電子字典都找不到
        res = requests.get(url.format(token1, q[k].encode('utf-8')), headers = header)
    #     print res.text


        #把facebook中找不到的興趣刪除,用新的dict裝起來

        if (len(json.loads(res.text)['data'])==0):
            pass
        else:    
            actualName = json.loads(res.text)['data'][0]['name']
            if actualName.encode('utf-8')!=q[k].encode('utf-8'):
                pass
            else:
            #.encode('utf-8')
                qid = json.loads(res.text)['data'][0]['id']
#                 print actualName
                newq[qid]=actualName 
#                 print (actualName)
            # print qid    
    

In [2]:
#找教育程度的人數  9

ch={'2':u'高中','3':u'大專院校','4':u'研究所'}   
def edu_info():
    
    for ele2 in ch.keys():
    #     print ele2
        final_text=text3['9']['data'][ele2]
        print ageMin,ageMax,gender[i],qid,name_interest,ch[ele2],final_text['audience']['total']
   


In [3]:
#找職稱  15
def emp_name():

    for ele in text3['15']['data']:
        final_text=text3['15']['data'][ele]
        print ageMin,ageMax,gender[i],qid,name_interest ,final_text['title'],final_text['audience']['total']

In [4]:
#感情狀況 27
relationship={'1':u'單身','2':u'穩定交往中','4':u'已婚','5':u'已訂婚'}
def relation():
    for ele in relationship.keys():
        final_text=text3['27']['data'][ele]
        print ageMin,ageMax,gender[i],qid,name_interest ,relationship[ele],final_text['audience']['total']